In [25]:
from open3d import utility
from open3d import geometry
from open3d import io
from open3d import visualization
import itertools as it
import trimesh
import numpy as np
import math
import glob
import random
from ipywidgets import FloatProgress
from IPython.display import display

searching for scad in: /Library/Frameworks/Python.framework/Versions/3.7/bin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/anaconda3/envs/custom/bin:/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Applications/OpenSCAD.app/Contents/MacOS


In [3]:
class Box:
    
    def __init__(self, points, grid_idx):
        self.points = points
        self.adjacent_boxes = []
        self.grid_idx = grid_idx 

class Grid:
    
    def __init__(self, points, density):
        self.points = points
        self.density = density
        self.boxes = []
        self.nonempty_boxes = []
        self.box_dimensions = None
        self.grid_dimensions = None
        self.total_boxes = None
        self.get_box_dimensions()
        self.create_grid()
    
    def get_box_dimensions(self):
        pcd = geometry.PointCloud()
        pcd.points = utility.Vector3dVector(self.points)
        tree = geometry.KDTreeFlann(pcd)
        box_boundaries = []
        for point in self.points:
            [_, neighbors_idx, _] = tree.search_knn_vector_3d(point, self.density)
            neighbor_points = []
            for idx in neighbors_idx:
                neighbor_points.append(self.points[idx])
            neighbor_points = np.asarray(neighbor_points)
            neighbor_boundary = neighbor_points.max(axis=0) - neighbor_points.min(axis=0)
            box_boundaries.append(neighbor_boundary)
        box_dimensions = np.mean(np.asarray(box_boundaries), axis=0)
        bounding_box = pcd.get_max_bound() - pcd.get_min_bound()
        grid_dimensions = (bounding_box/box_dimensions).astype(np.int64)
        box_dimensions = bounding_box/grid_dimensions
        total_boxes = grid_dimensions[0] * grid_dimensions[1] * grid_dimensions[2]
        self.box_dimensions = box_dimensions
        self.grid_dimensions = grid_dimensions
        self.total_boxes = total_boxes
        
    def get_nonempty_boxes(self):
        nonempty_boxes = []
        for box in self.boxes:
            if len(box.points) > 2:
                nonempty_boxes.append(box)
        self.nonempty_boxes = nonempty_boxes
    
    def get_adjacent_boxes(self):
        for current_box in self.nonempty_boxes:
            adjacent_boxes = []
            idx_y, idx_z, idx_x = current_box.grid_idx
            for i in range(-1,2):
                for j in range(-1,2):
                    for k in range(-1,2):
                        if not (i == 0 and j == 0 and k == 0):
                            for box in self.nonempty_boxes:
                                if box.grid_idx == (idx_y+i, idx_z+j, idx_x+k):
                                    adjacent_boxes.append(box)
            current_box.adjacent_boxes = adjacent_boxes
        
    def create_grid(self):
        print('|Starting Grid|\n')
        print(f'Number of Neighbors: {self.density}')
        print(f'Number of Boxes: {self.total_boxes}')
        print(f'Grid Dimensions: {self.grid_dimensions}')
        print(f'Box Dimensions: {self.box_dimensions}')
        
        points = self.points
        min_x, min_y, min_z = points.min(axis=0)
        current_x, current_y, current_z = min_x, min_y, min_z
        grid_x, grid_y, grid_z = self.grid_dimensions
        box_x, box_y, box_z = self.box_dimensions
        
        boxes = []

        f = FloatProgress(min=0, max=self.total_boxes)
        display(f)

        for idx_y in range(grid_y):
            interval_y = points[points[:,1] < current_y + box_y]
            interval_y = interval_y[interval_y[:,1] >= current_y]
            for idx_z in range(grid_z):
                interval_z = interval_y[interval_y[:,2] < current_z + box_z]
                interval_z = interval_z[interval_z[:,2] >= current_z]
                for idx_x in range(grid_x):
                    interval_x = interval_z[interval_z[:,0] < current_x + box_x]
                    interval_x = interval_x[interval_x[:,0] >= current_x]
                    grid_idx = (idx_y,idx_z,idx_x)
                    box_points = []
                    for point in interval_x:
                        box_points.append(Point(point))
                    current_box = Box(box_points, grid_idx)
                    boxes.append(current_box)
                    current_x += box_x
                    f.value += 1
                current_x = min_x
                current_z += box_z
            current_z = min_z
            current_y += box_y
        self.boxes = boxes
        self.get_nonempty_boxes()
        self.get_adjacent_boxes()
        print('\n|Grid Complete|')

In [4]:
class Point:
    
    def __init__(self, position):
        self.x, self.y, self.z = position
        self.idx = None
        self.state = 0
        
    def __eq__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return self.x == other.x and self.y == other.y and self.z == other.z
    
    def __sub__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return (self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __add__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return (self.x + other.x, self.y + other.y, self.z + other.z)
    
    def position(self):
        return (self.x, self.y, self.z)
    
    
class Edge:
    
    def __init__(self, vertices):
        self.vertices = vertices
        self.state = 0
    
    def __eq__(self, other):
        if not isinstance(other, Edge): return NotImplemented
        for vertex in other.vertices:
            if not vertex in self.vertices:
                return False
        return True

    
class Triangle:
    
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
    
    def angle_fitness(self, vertices):
        v1, v2, v3 = vertices
        for vertex in self.vertices:
            if not vertex in vertices:
                v0 = vertex
        u = Vector(v0 - v1)
        v = Vector(v2 - v1)
        n1 = v.cross(u)
        u = Vector(v3 - v1)
        n2 = u.cross(v)
        return abs(np.tan(n1.angle(n2)))
    
    def expanding_vector(self, edge):
        v1, v2 = edge.vertices
        for vertex in self.vertices:
            if not vertex in edge.vertices:
                v3 = vertex
        w = Vector(v3 - v1)
        v = Vector(v2 - v1)
        v = v.unit_vector()
        e = v * v.dot(w) - w
        return e.unit_vector()
        

class Vector:
    
    def __init__(self, vector):
        self.x, self.y, self.z = vector
    
    def __sub__(self, other):
        return Vector((self.x - other.x, self.y - other.y, self.z - other.z))
    
    def __truediv__(self, constant):
        return Vector((self.x / constant, self.y / constant, self.z / constant))
    
    def __mul__(self, constant):
        return Vector((self.x * constant, self.y * constant, self.z * constant))
    
    def dot(self, other):
        return self.x*other.x + self.y*other.y + self.z*other.z
    
    def cross(self, other):
        x = self.y*other.z-self.z*other.y
        y = self.z*other.x-self.x*other.z
        z = self.x*other.y-self.y*other.x
        return Vector((x, y, z))
    
    def length(self):
        return self.dot(self)**0.5
    
    def angle(self, other):
        cosine_angle = self.dot(other) / (self.length()*other.length())
        return np.arccos(cosine_angle)
    
    def unit_vector(self):
        return self / self.length()
    
    def position(self):
        return (self.x, self.y, self.z)

    
class PointCloud:
    
    def __init__(self):
        self.points = []
        self.grid = None
        self.pcd = None
        
    def read(self, filedir):
        try:
            self.pcd = io.read_point_cloud(filedir)
            self.points = np.asarray(self.pcd.points)
        except OSError:
            print('File Directory does not exist')
        
    def rotate(self, angle):
        points = self.points
        x = points[:,0]
        y = points[:,2] + abs(points[:,2].min())
        z = points[:,1]
        points = np.array([x, y, z]).transpose()
        angle = math.radians(angle)
        theta = [0,angle,0]
        r_x = np.array([[1,0,0],[0,math.cos(theta[0]),-math.sin(theta[0])],[0,math.sin(theta[0]),math.cos(theta[0])]])
        r_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])],[0,1,0],[-math.sin(theta[1]),0,math.cos(theta[1])]])
        r_z = np.array([[math.cos(theta[2]),-math.sin(theta[2]),0],[math.sin(theta[2]),math.cos(theta[2]),0],[0,0,1]])   
        r = np.dot(r_z, np.dot(r_y,r_x))
        self.points = np.matmul(points, r)
        
    def create_grid(self, density=20):
        self.grid = Grid(self.points, density)

In [5]:
folderdir = '/Users/nickf/Desktop/Custom-Fit/Data/Body-Datasets/Caesar-Data/CSRFM_PCD/'
filedirs = glob.glob(f'{folderdir}/*.ply')
file_num = 10
point_cloud = PointCloud()
point_cloud.read(filedirs[file_num])
point_cloud.rotate(232)

In [6]:
point_cloud.create_grid(density=20)

|Starting Grid|

Number of Neighbors: 20
Number of Boxes: 2958
Grid Dimensions: [17 29  6]
Box Dimensions: [49.88368497 61.93748647 47.78697507]


FloatProgress(value=0.0, max=2958.0)


|Grid Complete|


In [48]:
def visual(geometries, renderfile='/Users/nickf/Desktop/Custom-Fit/Scripts/render.json'):
    render = visualization.RenderOption()
    render.load_from_json(renderfile)
    visualization.draw_geometries(geometries)

def create_pcd(points):
    pcd = geometry.PointCloud()
    pcd.points = utility.Vector3dVector(points)
    return pcd

def create_mesh(vertices, triangles):
    mesh = geometry.TriangleMesh()
    mesh.vertices = utility.Vector3dVector(vertices)
    mesh.triangles = utility.Vector3iVector(triangles)
    return mesh

    
def estimate_normals(pcd, knn_factor=3):
    knn = int(len(pcd.points) / knn_factor)
    if knn < 2:
        knn = 2
    geometry.estimate_normals(pcd, geometry.KDTreeSearchParamKNN(knn))
    pcd.normalize_normals()
    points = np.asarray(pcd.points)
    tree = geometry.KDTreeFlann(pcd)
    for idx in range(len(points)):
        [_, neighbors_idx, _] = tree.search_knn_vector_3d(points[idx], knn+1)
        edges = list(it.permutations(neighbors_idx[1:], 2))
        triangles = np.array([(idx, edge[0], edge[1]) for edge in edges])
        mesh = create_mesh(points, triangles)
        mesh.compute_triangle_normals()
        return mesh
            
            

In [49]:
box_idx = 0
current_box = point_cloud.grid.nonempty_boxes[box_idx]
points = np.array([point.position() for point in current_box.points])
pcd = create_pcd(points)

In [52]:
mesh = estimate_normals(pcd)
lineset = geometry.create_line_set_from_triangle_mesh(mesh)
visual([pcd, mesh, lineset])

In [57]:
visual([create_pcd(np.asarray(mesh.triangle_normals))])

In [63]:
geometry.estimate_normals(point_cloud.pcd, geometry.KDTreeSearchParamKNN(10))
point_cloud.pcd.normalize_normals()
visual([point_cloud.pcd])

In [125]:
voxelgrid = geometry.create_surface_voxel_grid_from_point_cloud(point_cloud.pcd, voxel_size=50)
voxelgrid.voxels[0].grid_index

array([11,  3, 25], dtype=int32)

In [128]:

voxelgrid.get_voxel(np.asarray(point_cloud.pcd.points)[0])

array([ 5,  6, 16], dtype=int32)

In [145]:
import mcubes
vertices, triangles = mcubes.marching_cubes(u, 0)

In [146]:
mesh = create_mesh(vertices, triangles)
mesh.compute_vertex_normals()
lineset = geometry.create_line_set_from_triangle_mesh(mesh)
visual([mesh, lineset])

In [165]:
x, y, z = points.T

In [ ]:
X, Y, Z = np.meshgrid(x,y,z)